First, we import the relevant packages and make our API calls. (EXTRACT PHASE)

In [1]:
import requests
import config
import pandas as pd
import matplotlib.pyplot as plt 
#% matpltlib inline

key = config.key

api_PAY = f'https://api.census.gov/data/2018/abscs?get=NAME,EMP,PAYANN&for=state:*&NAICS2017=54&key={key}'
api_AI_use = f'https://api.census.gov/data/2018/abstcb?get=NAME,TECHUSE_LABEL,FIRMPDEMP,FIRMPDEMP_PCT,EMP&for=state:*&NAICS2017=54&TECHUSE=T1E03B01&TECHUSE=T1E03B99&key={key}'
api_CLOUD_use = f'https://api.census.gov/data/2018/abstcb?get=NAME,TECHUSE_LABEL,FIRMPDEMP,FIRMPDEMP_PCT,EMP&for=state:*&NAICS2017=54&TECHUSE=T2E03B01&TECHUSE=T2E03B99&key={key}'
api_AI_sell= f'https://api.census.gov/data/2018/abstcb?get=NAME,TECHSELL_LABEL,FIRMPDEMP,FIRMPDEMP_PCT,EMP&for=state:*&NAICS2017=54&TECHSELL=T1E20J01&TECHSELL=T1E20J02&key={key}'
api_CLOUD_sell =  f'https://api.census.gov/data/2018/abstcb?get=NAME,TECHSELL_LABEL,FIRMPDEMP,FIRMPDEMP_PCT,EMP&for=state:*&NAICS2017=54&TECHSELL=T2E20J01&TECHSELL=T2E20J02&key={key}'


Next, we load the data in Pandas DF's for cleaning and transforming. (LOAD PHASE)

In [2]:
#Used to determine 'avg pay', defined as payann/emp
pay_data = requests.get(api_PAY).json()
pay_df = pd.DataFrame(pay_data[1:], columns=pay_data[0])

#trends in aiusage across states: companies that did not use (T1E03B01), and companies that totally use it (T1E03B99)
ai_use_data = requests.get(api_AI_use).json()
ai_use_df = pd.DataFrame(ai_use_data[1:], columns=ai_use_data[0])

#trends in cloud usage across states: companies that did not use (T2E03B01), companies that totally use it (T2E03B99)
cloud_use_data = requests.get(api_CLOUD_use).json()
cloud_use_df = pd.DataFrame(cloud_use_data[1:], columns=cloud_use_data[0])

#trends in companies wrt ai based tech: companies that do sell (T1E20J01), companies that don't (T1E20J02)
ai_sell_data = requests.get(api_AI_sell).json()
ai_sell_df = pd.DataFrame(ai_sell_data[1:], columns=ai_sell_data[0])

#trends in companies wrt cloud-based tech: companies that do sell (T2E20J01), companies that don't (T2E20J02)
cloud_sell_data = requests.get(api_CLOUD_sell).json()
cloud_sell_df = pd.DataFrame(cloud_sell_data[1:], columns=cloud_sell_data[0])


Next, we clean and transform the data for practical use. First, we filter the rows according to their codes, and then we re-merge the filtered tables into combined tables for ease of comparison. Additionally, we remove unneeded columns, retype numerical data, and relabel the columns for easy use. (TRANSFORM PHASE)

In [3]:
#pay_df transforming --- adding column called AvgPay, defined as payann/emp
pay_df['EMP'] = pd.to_numeric(pay_df['EMP'])
pay_df['PAYANN'] = pd.to_numeric(pay_df['PAYANN'])
pay_df['AvgPay'] = pay_df['PAYANN']/pay_df['EMP']
pay_df = pay_df[['NAME','EMP','PAYANN','AvgPay']]
pay_df.rename(columns={"NAME":'State', 'EMP':'TotalEmployees', 'PAYANN':'AnnualPay'}, inplace=True)

#ai_use_df transforming, filtering out ai users and non-users from ai-users and merging them into one cleaner table
ai_users = ai_use_df[ai_use_df['TECHUSE'] == 'T1E03B99']
ai_nonusers = ai_use_df[ai_use_df['TECHUSE'] == 'T1E03B01']
ai_use_merge = ai_users.merge(ai_nonusers, on='NAME', how='inner', suffixes=('_user', '_nonuser'))
ai_use_merge = ai_use_merge[['NAME','FIRMPDEMP_user','FIRMPDEMP_PCT_user','EMP_user','FIRMPDEMP_nonuser','FIRMPDEMP_PCT_nonuser','EMP_nonuser']]
ai_use_merge.rename(columns={'NAME':'State', 'FIRMPDEMP_user':'TotalFirmsUsingAI', 'FIRMPDEMP_PCT_user':'PercentFirmsUsingAI','EMP_user':'TotalEmployeesUsingAI', 'FIRMPDEMP_nonuser':'TotalFirmsNoAI', 'FIRMPDEMP_PCT_nonuser':'PercentFirmsNoAI', 'EMP_nonuser':'TotalEmployeesNoAI'}, inplace=True)
ai_use_merge[['TotalFirmsUsingAI', 'PercentFirmsUsingAI', 'TotalEmployeesUsingAI', 'TotalFirmsNoAI', 'PercentFirmsNoAI', 'TotalEmployeesNoAI']] = ai_use_merge[['TotalFirmsUsingAI', 'PercentFirmsUsingAI', 'TotalEmployeesUsingAI', 'TotalFirmsNoAI', 'PercentFirmsNoAI', 'TotalEmployeesNoAI']].apply(pd.to_numeric)


#cloud_use_df transforming, filtering out cloud users and non-users from cloud-users and merging them into one cleaner table
cloud_users = cloud_use_df[cloud_use_df['TECHUSE'] == 'T2E03B99']
cloud_nonusers = cloud_use_df[cloud_use_df['TECHUSE'] == 'T2E03B01']
cloud_use_merge = cloud_users.merge(cloud_nonusers, on='NAME', how='inner', suffixes=('_user', '_nonuser'))
cloud_use_merge = cloud_use_merge[['NAME','FIRMPDEMP_user','FIRMPDEMP_PCT_user','EMP_user','FIRMPDEMP_nonuser','FIRMPDEMP_PCT_nonuser','EMP_nonuser']]
cloud_use_merge.rename(columns={'NAME':'State', 'FIRMPDEMP_user':'TotalFirmsUsingCloud', 'FIRMPDEMP_PCT_user':'PercentFirmsUsingCloud','EMP_user':'TotalEmployeesUsingCloud', 'FIRMPDEMP_nonuser':'TotalFirmsNoCloud', 'FIRMPDEMP_PCT_nonuser':'PercentFirmsNoCloud', 'EMP_nonuser':'TotalEmployeesNoCloud'}, inplace=True)
cloud_use_merge[['TotalFirmsUsingCloud', 'PercentFirmsUsingCloud', 'TotalEmployeesUsingCloud', 'TotalFirmsNoCloud', 'PercentFirmsNoCloud', 'TotalEmployeesNoCloud']] = cloud_use_merge[['TotalFirmsUsingCloud', 'PercentFirmsUsingCloud', 'TotalEmployeesUsingCloud', 'TotalFirmsNoCloud', 'PercentFirmsNoCloud', 'TotalEmployeesNoCloud']].apply(pd.to_numeric)


#ai_sell_df transforming, filtering out companies that sell and companies that don't sell ai-based tech and merging into one cleaner table
ai_sellers = ai_sell_df[ai_sell_df['TECHSELL'] == 'T1E20J01']
ai_nonsellers = ai_sell_df[ai_sell_df['TECHSELL'] == 'T1E20J02']
ai_sell_merge = ai_sellers.merge(ai_nonsellers, on='NAME', how='inner', suffixes=('_seller', '_nonseller'))
ai_sell_merge = ai_sell_merge[['NAME','FIRMPDEMP_seller','FIRMPDEMP_PCT_seller','EMP_seller','FIRMPDEMP_nonseller','FIRMPDEMP_PCT_nonseller','EMP_nonseller']]
ai_sell_merge.rename(columns={'NAME':'State', 'FIRMPDEMP_seller':'TotalFirmsSellingAI', 'FIRMPDEMP_PCT_seller':'PercentFirmsSellingAI','EMP_seller':'TotalEmployeesSellingAI', 'FIRMPDEMP_nonseller':'TotalFirmsNotSellingAI', 'FIRMPDEMP_PCT_nonseller':'PercentFirmsNotSellingAI', 'EMP_nonseller':'TotalEmployeesNotSellingAI'}, inplace=True)
ai_sell_merge[['TotalFirmsSellingAI', 'PercentFirmsSellingAI', 'TotalEmployeesSellingAI', 'TotalFirmsNotSellingAI', 'PercentFirmsNotSellingAI', 'TotalEmployeesNotSellingAI']] = ai_sell_merge[['TotalFirmsSellingAI', 'PercentFirmsSellingAI', 'TotalEmployeesSellingAI', 'TotalFirmsNotSellingAI', 'PercentFirmsNotSellingAI', 'TotalEmployeesNotSellingAI']].apply(pd.to_numeric)

#cloud_sell_df transforming, filtering out comanies that sell and don't sell colud-based tech and merging into one cleaner table
cloud_sellers = cloud_sell_df[cloud_sell_df['TECHSELL'] == 'T2E20J01']
cloud_nonsellers = cloud_sell_df[cloud_sell_df['TECHSELL'] == 'T2E20J02']
cloud_sell_merge = cloud_sellers.merge(cloud_nonsellers, on='NAME',  how='inner', suffixes=('_seller', '_nonseller'))
cloud_sell_merge = cloud_sell_merge[['NAME','FIRMPDEMP_seller','FIRMPDEMP_PCT_seller','EMP_seller','FIRMPDEMP_nonseller','FIRMPDEMP_PCT_nonseller','EMP_nonseller']]
cloud_sell_merge.rename(columns={'NAME':'State', 'FIRMPDEMP_seller':'TotalFirmsSellingCloud', 'FIRMPDEMP_PCT_seller':'PercentFirmsSellingCloud','EMP_seller':'TotalEmployeesSellingCloud', 'FIRMPDEMP_nonseller':'TotalFirmsNotSellingCloud', 'FIRMPDEMP_PCT_nonseller':'PercentFirmsNotSellingCloud', 'EMP_nonseller':'TotalEmployeesNotSellingCloud'}, inplace=True)
cloud_sell_merge[['TotalFirmsSellingCloud', 'PercentFirmsSellingCloud', 'TotalEmployeesSellingCloud', 'TotalFirmsNotSellingCloud', 'PercentFirmsNotSellingCloud', 'TotalEmployeesNotSellingCloud']] = cloud_sell_merge[['TotalFirmsSellingCloud', 'PercentFirmsSellingCloud', 'TotalEmployeesSellingCloud', 'TotalFirmsNotSellingCloud', 'PercentFirmsNotSellingCloud', 'TotalEmployeesNotSellingCloud']].apply(pd.to_numeric)


Finally, we save the transformed dataframes into CSV files so that we can easily access the data in other Python scripts.

In [ ]:
pay_df.to_csv('pay_data.csv')
ai_use_merge.to_csv('ai_use.csv')
cloud_use_merge.to_csv('cloud_use.csv')
ai_sell_merge.to_csv('ai_sell.csv')
cloud_sell_merge.to_csv('cloud_sell.csv')